# Notebook 06: End-to-End Demo

Full pipeline execution on N PDF versions:
1. Load PDFs
2. Layer 0: Preprocess images
3. Layer 1: Classify pages
4. Layer 2: Extract elements
5. Layer 3: Compare versions (sequential pairwise)
6. Layer 4: Agentic verification (if LLM available)
7. Layer 5: Generate report

Includes performance profiling per layer.

In [ ]:
import sys
sys.path.insert(0, '..')

import time
import yaml
import json
import fitz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from src.preprocessing.image_preprocessor import ImagePreprocessor
from src.classifier.page_classifier import PageClassifier
from src.comparators.multi_version import MultiVersionComparator

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('All modules loaded. Ready for end-to-end demo.')

In [ ]:
# Configuration
PDF_VERSIONS = [
    'sample_pdfs/v1.pdf',
    'sample_pdfs/v2.pdf',
    'sample_pdfs/v3.pdf',
]

existing = [p for p in PDF_VERSIONS if Path(p).exists()]
print(f'Found {len(existing)} PDF versions to compare.')

if len(existing) < 2:
    print('\nPlease add at least 2 PDF files to sample_pdfs/ to run the demo.')
    print('Example: copy two versions of the same document as v1.pdf and v2.pdf')

## Full Pipeline Run with Profiling

In [ ]:
timings = {}

if len(existing) >= 2:
    # ---- Layer 0-2: Parse all versions ----
    t0 = time.time()
    comparator = MultiVersionComparator(pdf_paths=existing, config=config)
    comparator._parse_all_versions()
    timings['Parsing (Layers 0-2)'] = time.time() - t0
    print(f'Parsing completed in {timings["Parsing (Layers 0-2)"]:.2f}s')
    
    for path, doc in comparator.parsed_cache.items():
        print(f'  {Path(path).name}: {doc.summary()}')
    
    # ---- Layer 3: Compare pairs ----
    t0 = time.time()
    report = comparator.run()
    timings['Comparison (Layer 3)'] = time.time() - t0
    print(f'\nComparison completed in {timings["Comparison (Layer 3)"]:.2f}s')
    
    # ---- Results ----
    print(f'\n{"=" * 60}')
    print(f'RESULTS')
    print(f'{"=" * 60}')
    print(f'Versions compared: {report.version_labels}')
    print(f'Total changes: {report.total_changes}')
    print(f'Changes by type: {report.changes_by_type}')
    
    for diff in report.pair_diffs:
        print(f'\n  {diff.version_label}:')
        print(f'    Changes: {diff.change_counts}')
        print(f'    Pages added: {diff.pages_added}')
        print(f'    Pages removed: {diff.pages_removed}')
else:
    print('Skipping - need at least 2 PDFs.')

## Performance Profile

In [ ]:
if timings:
    fig, ax = plt.subplots(figsize=(10, 5))
    labels = list(timings.keys())
    values = list(timings.values())
    bars = ax.barh(labels, values, color=['steelblue', 'coral'])
    ax.set_xlabel('Time (seconds)')
    ax.set_title('Pipeline Execution Time by Layer')
    for bar, val in zip(bars, values):
        ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
                f'{val:.2f}s', va='center')
    plt.tight_layout()
    plt.show()
    
    total = sum(values)
    print(f'\nTotal pipeline time: {total:.2f}s')
    if len(existing) >= 2:
        print(f'Average per PDF: {total / len(existing):.2f}s')

## Detailed Change Inspection

In [ ]:
if len(existing) >= 2 and report:
    print('\n=== Detailed Text Changes ===')
    for diff in report.pair_diffs:
        print(f'\n--- {diff.version_label} ---')
        for change in diff.text_changes[:10]:  # Show first 10
            print(f'  [{change.change_type.value}] {change.element_id}')
            if change.old_text:
                snippet = change.old_text[:60].replace('\n', ' ')
                print(f'    Old: {snippet}...' if len(change.old_text) > 60 else f'    Old: {snippet}')
            if change.new_text:
                snippet = change.new_text[:60].replace('\n', ' ')
                print(f'    New: {snippet}...' if len(change.new_text) > 60 else f'    New: {snippet}')

In [ ]:
print('\n' + '=' * 60)
print('End-to-end demo complete!')
print('=' * 60)
print('\nNext steps:')
print('  1. Add real PDF versions to sample_pdfs/')
print('  2. Install Ollama for LLM verification (notebook 04)')
print('  3. Install Docling/Surya for advanced extraction (notebook 02)')
print('  4. Run individual layer notebooks for detailed exploration')